In [26]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [27]:
import nltk

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [28]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [29]:
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [30]:
df.shape

(11314, 3)

In [31]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [32]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [33]:
# Build the bigram and trigram models
# min_count (float, optional) – Ignore all words and bigrams with total collected count lower than this value


bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [34]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

In [35]:

nlp = spacy.load("en_core_web_sm") # shortcut link

C:\Users\Sky\anaconda3\envs\tensorflow_env\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [36]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_posting', 'host', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [38]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]]


In [39]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('neighborhood', 1),
  ('nntp_posting', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('sport', 1),
  ('thank', 1),
  ('thing', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [40]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
'''
chunksize (int, optional) – Number of documents to be used in each training chunk.

passes (int, optional) – Number of passes through the corpus during training.

update_every (int, optional) – Number of documents to be iterated through for each update. Set to 0 for batch learning, > 1 for online iterative learning.
'''

In [41]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.051*"state" + 0.037*"right" + 0.036*"law" + 0.031*"government" + '
  '0.019*"people" + 0.018*"country" + 0.016*"force" + 0.016*"member" + '
  '0.015*"israeli" + 0.014*"public"'),
 (1,
  '0.043*"car" + 0.034*"high" + 0.032*"buy" + 0.028*"price" + 0.026*"sell" + '
  '0.025*"cost" + 0.022*"low" + 0.022*"power" + 0.022*"player" + 0.022*"new"'),
 (2,
  '0.027*"go" + 0.019*"good" + 0.018*"time" + 0.017*"year" + 0.016*"think" + '
  '0.016*"well" + 0.015*"s" + 0.015*"make" + 0.014*"see" + 0.014*"get"'),
 (3,
  '0.089*"disk" + 0.083*"display" + 0.070*"server" + 0.062*"bus" + '
  '0.061*"family" + 0.053*"box" + 0.047*"recommend" + 0.036*"should" + '
  '0.035*"motif" + 0.031*"popular"'),
 (4,
  '0.062*"report" + 0.055*"science" + 0.040*"research" + 0.035*"study" + '
  '0.024*"scientific" + 0.021*"activity" + 0.021*"operate" + 0.018*"external" '
  '+ 0.018*"step" + 0.016*"development"'),
 (5,
  '0.176*"com" + 0.113*"distribution" + 0.039*"frame" + 0.031*"locate" + '
  '0.027*"enable" + 0

In [42]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -13.581087844236793

Coherence Score:  0.46819574585357115


In [17]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.330985  0.196612       1        1  25.956228
8      0.340913  0.138003       2        1  23.782513
10     0.319445  0.066125       3        1  17.911565
11     0.181398 -0.067269       4        1   6.311454
19     0.131170 -0.302102       5        1   4.819918
0      0.096038 -0.033113       6        1   4.541691
1      0.100616 -0.140510       7        1   4.317426
17     0.027068  0.082730       8        1   2.124227
18     0.030376 -0.235896       9        1   1.804285
14    -0.053984  0.001651      10        1   1.742891
4     -0.032616 -0.066717      11        1   1.730842
13    -0.103123  0.020759      12        1   1.006527
12    -0.143628  0.029257      13        1   0.796850
3     -0.151617  0.039968      14        1   0.690116
9     -0.153234  0.038340      15        1   0.626475
16    -0.158643  0.038954      16        1   0.621050
6     -0.178465  0.047240      17        1   0.453559
5     -0.188574  0.047522      18        1   0.299659
15    -0.192938  0.048014      19        1   0.249688
7     -0.201188  0.050433      20        1   0.213036, topic_info=           Term          Freq         Total Category  logprob  loglift
17         line  13771.000000  13771.000000  Default   30.000  30.0000
188       write  13842.000000  13842.000000  Default   29.000  29.0000
169     article   9059.000000   9059.000000  Default   28.000  28.0000
186      system   4654.000000   4654.000000  Default   27.000  27.0000
161         use   5918.000000   5918.000000  Default   26.000  26.0000
...         ...           ...           ...      ...      ...      ...
3385       wire      0.035887      1.230027  Topic20  -11.106   2.6170
3300     accord      0.035887      1.230012  Topic20  -11.106   2.6171
4232     expert      0.035887      1.229964  Topic20  -11.106   2.6171
8007  terrorist      0.035887      1.229996  Topic20  -11.106   2.6171
3448    capable      0.035887      1.229982  Topic20  -11.106   2.6171

[865 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
815       6  0.993768    abuse
1226      2  0.998430   accept
77        5  0.750333   access
77        9  0.247778   access
565      11  0.990220  achieve
...     ...       ...      ...
383      16  0.087799   writer
35        1  0.997248     year
35        8  0.002456     year
558       1  0.998881    young
1994      8  0.992757     zone

[1061 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 11, 12, 20, 1, 2, 18, 19, 15, 5, 14, 13, 4, 10, 17, 7, 6, 16, 8])

In [25]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.099922  0.097484       1        1  13.363887
8      0.098198  0.068645       2        1  12.630335
0      0.102474  0.107179       3        1  11.795379
2      0.071737 -0.031519       4        1   9.554378
14     0.062837 -0.071319       5        1   8.660430
19     0.039404 -0.119185       6        1   7.288291
16     0.077334  0.079438       7        1   6.380606
3      0.072338  0.151880       8        1   5.199253
4      0.048870 -0.012536       9        1   5.031432
12     0.018015 -0.162905      10        1   4.310159
18     0.019408 -0.008149      11        1   2.954584
15    -0.011687 -0.163205      12        1   2.537745
11     0.015785  0.026637      13        1   2.480174
1     -0.011856 -0.074043      14        1   1.907320
13    -0.020723 -0.006530      15        1   1.728454
6     -0.002650  0.065100      16        1   1.330874
9     -0.396862  0.096057      17        1   0.987705
5     -0.082036  0.001497      18        1   0.758002
17    -0.049704 -0.010772      19        1   0.748808
7     -0.150804 -0.033753      20        1   0.352183, topic_info=        Term          Freq         Total Category  logprob  loglift
3885      ax   7052.000000   7052.000000  Default  30.0000  30.0000
17      line  10542.000000  10542.000000  Default  29.0000  29.0000
633      key   2242.000000   2242.000000  Default  28.0000  28.0000
315     file   2637.000000   2637.000000  Default  27.0000  27.0000
85       bit   1729.000000   1729.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
110     hear     13.757836   1452.126161  Topic20  -5.6597   0.9896
12      host     12.841230   2665.297266  Topic20  -5.7287   0.3134
186   system     12.665521   3529.020398  Topic20  -5.7424   0.0189
84        be     12.661120   5886.422424  Topic20  -5.7428  -0.4931
27       see     11.387131   4439.839639  Topic20  -5.8488  -0.3171

[1642 rows x 6 columns], token_table=      Topic      Freq Term
term                      
7773      1  0.012370    _
7773      2  0.018555    _
7773      3  0.046386    _
7773      4  0.009277    _
7773      5  0.012370    _
...     ...       ...  ...
4618      8  0.008174  zip
4618      9  0.008174  zip
4618     10  0.686604  zip
4618     12  0.089912  zip
4618     15  0.130782  zip

[8205 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 9, 1, 3, 15, 20, 17, 4, 5, 13, 19, 16, 12, 2, 14, 7, 10, 6, 18, 8])

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)